In [ ]:
import ee, geetools

ee.Initialize()

In [ ]:
fc = (
    ee.FeatureCollection("FAO/GAUL/2015/level0")
    .filter(ee.Filter.inList("ADM0_CODE", [122, 237, 85]))
)
geom = fc.geetools.mergeGeometries()

In [ ]:
import geemap
m = geemap.Map()
m.addLayer(geom, {}, "Countries")
m

In [ ]:
vatican = ee.Geometry.Point([12.4534, 41.9033]).buffer(1)
fc = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_CODE', 110))
fc

In [ ]:
vatican = ee.Geometry.Point([12.4534, 41.9033]).buffer(1)
image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(vatican).first()
image = image.select(["B4", "B2"]).geetools.interpolateBands([0, 3000], [0, 30])
values = image.reduceRegion(ee.Reducer.mean(), vatican, 1)
print(values.getInfo())

In [ ]:
image.getInfo()

In [ ]:
values = image.reduceRegion(ee.Reducer.first(), vatican, 1)

In [ ]:
values.getInfo()

In [ ]:
import ee, geetools
from itertools import permutations

list([i for i in permutations([0,1], 2)])

In [ ]:
à

In [ ]:
new_fc = ee.FeatureCollection([geometryCol]).geetools.toPolygons()

In [ ]:
geometryCol.geetools.keepType("Point").getInfo()

In [1]:
import ee
import geetools

ee.Initialize()

point = ee.Geometry.Point([0,0])
prop1 = {'id': 1, 'prop_from_fc1': 'I am from fc1'}
prop2 = {'id': 1, 'prop_from_fc2': 'I am from fc2'}
fc1 = ee.FeatureCollection([ee.Feature(point, prop1)])
fc2 = ee.FeatureCollection([ee.Feature(point, prop2)])
joined = ee.Join.geetools.byProperty(fc1, fc2, 'id')
joined.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [0, 0]},
   'id': '0',
   'properties': {'id': 1,
    'prop_from_fc1': 'I am from fc1',
    'prop_from_fc2': 'I am from fc2'}}]}

In [ ]:
import ee
import geetools

ee.Initialize()

fc = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.inList("ADM0_CODE", [122, 237, 85]))
fc1 = fc.select(["ADM0_CODE", "ADM0_NAME"])
fc2 = fc.select(["ADM0_CODE", "STATUS"])
joined = ee.Join.geetools.byProperty(fc1, fc2, "ADM0_CODE")
joined.getInfo()

In [ ]:
import ee
ee.Initialize()

fc = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.inList("ADM0_CODE", [122, 237, 85]))
fc1 = fc.select(["ADM0_CODE", "ADM0_NAME"])
fc2 = fc.select(["ADM0_CODE", "STATUS"])

def join(primary, secondary, field, outer=False):


    Filter = ee.Filter.equals(leftField=field, rightField=field)
    join = ee.Join.saveFirst(matchKey="match", outer=outer)
    joined = join.apply(primary, secondary, Filter)

    def cleanJoin(feat):
        primaryProp = feat.propertyNames().remove("match")
        secondaryProp = ee.Feature(feat.get("match")).toDictionary()
        return feat.select(primaryProp).setMulti(secondaryProp)

    return ee.FeatureCollection(joined.map(cleanJoin))

print(join(fc1, fc2, "ADM0_CODE").getInfo())